In [ ]:
import pandas as pd
import numpy as np 
import plotly as pl

In [ ]:
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, Dense, LSTM
from tensorflow.keras import Sequential

In [ ]:
with open("examplemutantpayloads.csv", "r", encoding="latin1") as file:
    raw_data = file.read()

In [ ]:
exploits = ['aix/ppc/shell_find_port', 'aix/ppc/shell_interact', 'aix/ppc/shell_reverse_tcp', 'android/meterpreter/reverse_http', 'aix/ppc/shell_bind_tcp_small', 'aix/ppc/shell_find_port', 'aix/ppc/shell_interact', 'aix/ppc/shell_reverse_tcp', 'android/meterpreter/reverse_http']

In [ ]:
data = []
for i in range(len(exploits)-1):
    data.append(raw_data[raw_data.index(exploits[i]): raw_data.index(exploits[i+1])])

data = [tuple(i.split(",")) + ("",) for i in data]
Y = np.random.randint(0, 2, (len(data), 2))

In [ ]:
Y, Y.shape, Y.dtype

(array([[0, 1],
        [0, 0],
        [0, 1],
        [0, 0],
        [1, 0],
        [1, 1],
        [0, 0],
        [0, 0]]), (8, 2), dtype('int64'))

In [ ]:
len(set([i for a in data for i in a[1]]))

200

In [ ]:
VOCAB_SIZE = 1000
encoder = TextVectorization(
    max_tokens=VOCAB_SIZE)

Normalize and tokenize the payloads

In [ ]:
train_dataset = [i[1] for i in data]
encoder.adapt(train_dataset)

Get vocabulary

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'øk\x87\x93ö\x11ú\x00\x85\x89\x07',
       'ëÚ\x08o\x9a\x89·¿×a¸öÔ\x08\x899»ëywì\x88êÏ',
       'ÚÑ½\x8a3©\x02Ùtôz1É±4\x83Â\x041j\x13\x03à',
       'ÚÊ¸\x076¼Ùtô3É±\x0f1c\x17\x83Ã\x04\x03dÔi6ç2Ë\x86e½Öy\x82\x8f¾\xaddïø¸çÀo¹è\xa0\x0f¹\x19ppº\x0eØðº',
       '½ª\x85ÇÛÈÙtô3É±8\x83ëü1k\x0e\x034¤g2µg\x9dÄ',
       '¸s\x0fåÐvÛ\x9drýùÕ\x1e\x06\xad\x11¼\x03Ý\x1e÷÷d®\x87ü©\x19±\x03va\x04ÙäÑ\x07aeÚ\x05\x1eapÛicòÖ²ËºÙô\x7f\x1bÚ',
       '±º0\x08w',
       '¦\x03Æ¬v\x07\x94\x12¢\x80\x16l\x08\x89sbjq§ã\x89z¶ìÖg\x10wácmØmpöy\x8d\x8c\x7f\x9d',
       '\x9dÊöò\x933Ýdx\x98Ç\x01vf³\x08m¾\x07ÏüÓ0\x02à\x10û',
       '\x8bt¢ghó\x12økÚ\x91¤rãf\x0e\x8b\x18\xa0µ«\x8bðk´Ëv³\x8atª\x88õ\x92ñ÷\x11\x9dÀ',
       'k÷\x88ã¡\x810nj\x8cÏ¦·É\x14é¹êqËd1Ú»\x9cj¸À\x8dÄ9±èÌ3ã\x019pû\x84úpûqÇ'],
      dtype='<U57')

Generate a RNN-LSTM Model to classify the payloads

In [ ]:
model = Sequential([
    encoder,
    Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True), # mask for variable length sequence
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'), # Rectified Linear Activation
    Dense(2)
])


In [ ]:
model.compile("adam", loss="binary_crossentropy", metrics=['accuracy'] )

In [ ]:
model.fit(np.asarray(train_dataset), Y, epochs=1, verbose=1, )

1/1 [==============================] - 8s 8s/step - loss: 3.8567 - accuracy: 0.7500
